In [1]:
# python
import os
import time
import json
from datetime import datetime
from tqdm.notebook import tqdm
import pandas as pd
from spotify_functions import *  # assumes helper functions exist

# Load CSV
artists_ids_df = pd.read_csv("artist_ids.csv")

# Backup file
backup_file = '../spotify_artists_backup.json'
if os.path.exists(backup_file):
    with open(backup_file, 'r') as f:
        artists_data = json.load(f)
    print(f"Resuming with {len(artists_data)} existing artists")
    # Safe processed_ids: only dicts and non-null ids, normalized to str
    processed_ids = {
        str(artist.get('artist_id')).strip()
        for artist in artists_data
        if isinstance(artist, dict) and artist.get('artist_id') is not None and not pd.isna(artist.get('artist_id'))
    }
else:
    artists_data = []
    processed_ids = set()

total_artists = len(artists_ids_df)
processed_count = len(artists_data)

REQUEST_DELAY = 2.0
SAVE_INTERVAL = 5

print(f"Processing {total_artists - processed_count} remaining artists out of {total_artists} total")

# Guard for access token
if 'access_token' not in globals() or not access_token:
    raise RuntimeError("`access_token` not defined. Define it before running or obtain a token from your auth flow.")

with tqdm(total=total_artists, initial=processed_count, desc='Artists', unit='artist') as pbar:
    for index, row in artists_ids_df.iterrows():
        artist_name = row.get('artist', '')
        raw_id = row.get('artist_id')

        # Print using loop index (1-based)
        # Normalize and skip missing IDs
        if pd.isna(raw_id):
            print(f"Skipping {artist_name} due to missing artist ID.")
            pbar.update(1)
            continue

        artist_id = str(raw_id).strip()

        # Skip if already processed
        if artist_id in processed_ids:
            print(f"Skipping {artist_name} (already processed)")
            pbar.update(1)
            continue

        print(f"[{index + 1}/{total_artists}] Processing Artist: {artist_name}, Spotify ID: {artist_id}")

        try:
            # Respect rate limits
            time.sleep(REQUEST_DELAY)

            # Get artist info
            artist_info = get_artist_info(artist_id, access_token)

            # Normalize artist_info structure
            if isinstance(artist_info, list) and artist_info:
                artist_info = artist_info[0]
            elif isinstance(artist_info, dict) and isinstance(artist_info.get('artists'), list) and artist_info['artists']:
                artist_info = artist_info['artists'][0]

            followers_count = None
            if isinstance(artist_info, dict):
                followers = artist_info.get('followers', {})
                if isinstance(followers, dict):
                    followers_count = followers.get('total')

            time.sleep(REQUEST_DELAY)

            # Get albums
            albums_resp = get_artist_albums(artist_id, access_token)
            if isinstance(albums_resp, dict) and isinstance(albums_resp.get('items'), list):
                albums_list = albums_resp['items']
            elif isinstance(albums_resp, list):
                albums_list = albums_resp
            else:
                albums_list = []

            album_names = [a.get('name') for a in albums_list if isinstance(a, dict)]
            album_release_dates = [a.get('release_date') for a in albums_list if isinstance(a, dict)]

            tracks = []
            for album in albums_list:
                album_id = album.get('id') if isinstance(album, dict) else None
                if not album_id:
                    continue

                time.sleep(REQUEST_DELAY)

                album_tracks_resp = get_album_tracks(album_id, access_token)
                if isinstance(album_tracks_resp, dict) and isinstance(album_tracks_resp.get('items'), list):
                    tracks_list = album_tracks_resp['items']
                elif isinstance(album_tracks_resp, list):
                    tracks_list = album_tracks_resp
                else:
                    tracks_list = []

                for track in tracks_list:
                    if isinstance(track, dict) and 'name' in track:
                        tracks.append(track['name'])

            # Append normalized artist data (use string id)
            artists_data.append({
                'artist': artist_name,
                'artist_id': artist_id,
                'followers_count': followers_count,
                'albums': album_names,
                'album_release_dates': album_release_dates,
                'tracks': tracks
            })

            # Mark processed and update counters
            processed_ids.add(artist_id)
            processed_count += 1

            print(f"Collected data for {artist_name}: {len(album_names)} albums, {len(tracks)} tracks, {followers_count} followers.")

            # Periodic save
            if processed_count % SAVE_INTERVAL == 0:
                with open(backup_file, 'w') as f:
                    json.dump(artists_data, f, indent=2)
                print(f"✓ Backup saved ({processed_count} artists)")

            pbar.update(1)

        except Exception as e:
            print(f"Error processing {artist_name}: {e}")
            # Save backup on error
            with open(backup_file, 'w') as f:
                json.dump(artists_data, f, indent=2)
            print(f"✓ Backup saved after error")

            # Rate limit handling
            if "429" in str(e) or "rate limit" in str(e).lower():
                wait_time = 60
                print(f"Rate limit detected. Waiting {wait_time} seconds...")
                time.sleep(wait_time)

            pbar.update(1)

# Final save
with open(backup_file, 'w') as f:
    json.dump(artists_data, f, indent=2)
print(f"\n✓ Final backup saved: {len(artists_data)} artists")

# Save to CSV
artists_data_df = pd.DataFrame(artists_data)
artists_data_df.to_csv('spotify_artists_data.csv', index=False)
print(f"✓ Saved to CSV: spotify_artists_data.csv")

Processing 5184 remaining artists out of 5184 total


Artists:   0%|          | 0/5184 [00:00<?, ?artist/s]

[1/5184] Processing Artist: Jeff Tweedy, Spotify ID: 2rDxtYUzTAYJJE3Bl3Z5IN


KeyboardInterrupt: 

##### NEW CODE

In [4]:
# language: python
import os
import time
import json
from datetime import datetime
from tqdm.notebook import tqdm
import pandas as pd
from spotify_functions import *  # helper functions: get_artist_info, get_artist_albums, get_album_tracks

VERBOSE = True

def status(msg):
    if VERBOSE:
        print(f"{datetime.now().isoformat(sep=' ', timespec='seconds')} — {msg}")

# Files
csv_file = "artist_ids.csv"
backup_file = "../spotify_artists_backup.json"
output_csv = "spotify_artists_data.csv"

# Load CSV
artists_ids_df = pd.read_csv(csv_file)

# Load or initialize backup & processed set (normalize ids to str)
if os.path.exists(backup_file):
    with open(backup_file, 'r') as f:
        artists_data = json.load(f)
    status(f"Resuming with {len(artists_data)} existing artists from {backup_file}")
    processed_ids = {
        str(item.get('artist_id')).strip()
        for item in artists_data
        if isinstance(item, dict) and item.get('artist_id') is not None and not pd.isna(item.get('artist_id'))
    }
else:
    artists_data = []
    processed_ids = set()
    status(f"No backup found at {backup_file}; starting fresh")

total_artists = len(artists_ids_df)
processed_count = len(artists_data)

REQUEST_DELAY = 2.0
SAVE_INTERVAL = 5

status(f"Will process {total_artists - processed_count} remaining artists (total {total_artists})")

# Guard: require access_token to be defined by the user/environment
if 'access_token' not in globals() or not access_token:
    raise RuntimeError("`access_token` not defined. Define it before running or obtain a token from your auth flow.")

with tqdm(total=total_artists, initial=processed_count, desc='Artists', unit='artist') as pbar:
    for index, row in artists_ids_df.iterrows():
        loop_start = time.perf_counter()
        artist_name = row.get('artist', '') or ''
        raw_id = row.get('artist_id')

        # Missing ID -> skip
        if pd.isna(raw_id):
            status(f"Skipping row {index + 1}: {artist_name} — missing `artist_id`")
            pbar.set_postfix({'stage': 'skipped', 'artist': artist_name})
            pbar.update(1)
            continue

        artist_id = str(raw_id).strip()
        if not artist_id:
            status(f"Skipping row {index + 1}: {artist_name} — empty `artist_id` after normalization")
            pbar.set_postfix({'stage': 'skipped', 'artist': artist_name})
            pbar.update(1)
            continue

        # Already processed -> skip
        if artist_id in processed_ids:
            status(f"Skipping [{index + 1}/{total_artists}] {artist_name} ({artist_id}) — already processed")
            pbar.set_postfix({'stage': 'already_processed', 'artist': artist_name})
            pbar.update(1)
            continue

        status(f"[{index + 1}/{total_artists}] Starting {artist_name} ({artist_id})")
        pbar.set_postfix({'stage': 'start', 'artist': artist_name, 'id': artist_id})

        try:
            # Fetch artist info
            pbar.set_postfix({'stage': 'fetching_artist', 'artist': artist_name})
            time.sleep(REQUEST_DELAY)
            artist_info = get_artist_info(artist_id, access_token)

            # normalize artist_info to a dict if wrapped
            if isinstance(artist_info, list) and artist_info:
                artist_info = artist_info[0]
            elif isinstance(artist_info, dict) and isinstance(artist_info.get('artists'), list) and artist_info['artists']:
                artist_info = artist_info['artists'][0]

            followers_count = None
            if isinstance(artist_info, dict):
                followers = artist_info.get('followers', {})
                if isinstance(followers, dict):
                    followers_count = followers.get('total')

            status(f"Fetched artist info: followers={followers_count}")
            pbar.set_postfix({'stage': 'fetched_artist', 'followers': followers_count})

            # Fetch albums
            pbar.set_postfix({'stage': 'fetching_albums', 'artist': artist_name})
            time.sleep(REQUEST_DELAY)
            albums_resp = get_artist_albums(artist_id, access_token)
            if isinstance(albums_resp, dict) and isinstance(albums_resp.get('items'), list):
                albums_list = albums_resp['items']
            elif isinstance(albums_resp, list):
                albums_list = albums_resp
            else:
                albums_list = []

            album_names = [a.get('name') for a in albums_list if isinstance(a, dict)]
            album_release_dates = [a.get('release_date') for a in albums_list if isinstance(a, dict)]
            status(f"Found {len(album_names)} albums for {artist_name}")
            pbar.set_postfix({'stage': 'fetched_albums', 'albums_found': len(album_names)})

            # Fetch tracks per album
            tracks = []
            for i, album in enumerate(albums_list, start=1):
                album_id = album.get('id') if isinstance(album, dict) else None
                if not album_id:
                    continue

                pbar.set_postfix({
                    'stage': 'fetching_album_tracks',
                    'artist': artist_name,
                    'album_idx': f"{i}/{len(albums_list)}",
                    'tracks_collected': len(tracks)
                })
                status(f"  [{artist_name}] Fetching tracks for album {i}/{len(albums_list)}")
                time.sleep(REQUEST_DELAY)
                album_tracks_resp = get_album_tracks(album_id, access_token)
                if isinstance(album_tracks_resp, dict) and isinstance(album_tracks_resp.get('items'), list):
                    tracks_list = album_tracks_resp['items']
                elif isinstance(album_tracks_resp, list):
                    tracks_list = album_tracks_resp
                else:
                    tracks_list = []

                for track in tracks_list:
                    if isinstance(track, dict) and 'name' in track:
                        tracks.append(track['name'])

                # update postfix after each album
                pbar.set_postfix({
                    'stage': 'fetching_album_tracks',
                    'artist': artist_name,
                    'album_idx': f"{i}/{len(albums_list)}",
                    'tracks_collected': len(tracks)
                })

            # Append normalized artist data
            artists_data.append({
                'artist': artist_name,
                'artist_id': artist_id,
                'followers_count': followers_count,
                'albums': album_names,
                'album_release_dates': album_release_dates,
                'tracks': tracks
            })

            processed_ids.add(artist_id)
            processed_count += 1

            elapsed = time.perf_counter() - loop_start
            status(f"Collected {len(album_names)} albums and {len(tracks)} tracks for {artist_name}. Followers: {followers_count}. Time: {elapsed:.2f}s")
            pbar.set_postfix({
                'stage': 'done',
                'artist': artist_name,
                'albums': len(album_names),
                'tracks': len(tracks),
                'followers': followers_count
            })

            # Periodic save
            if processed_count % SAVE_INTERVAL == 0:
                with open(backup_file, 'w') as f:
                    json.dump(artists_data, f, indent=2)
                status(f"✓ Backup saved ({processed_count} artists) to {backup_file}")
                pbar.set_postfix({'stage': 'backup_saved', 'saved': processed_count})

            pbar.update(1)

        except Exception as e:
            status(f"Error processing {artist_name}: {e}")
            pbar.set_postfix({'stage': 'error', 'artist': artist_name, 'error': str(e)[:120]})
            # Save backup on error
            with open(backup_file, 'w') as f:
                json.dump(artists_data, f, indent=2)
            status(f"✓ Backup saved after error ({len(artists_data)} artists) to {backup_file}")

            # Rate limit handling
            if "429" in str(e) or "rate limit" in str(e).lower():
                wait_time = 60
                status(f"Rate limit detected. Waiting {wait_time}s...")
                time.sleep(wait_time)

            pbar.update(1)

# Final save
with open(backup_file, 'w') as f:
    json.dump(artists_data, f, indent=2)
status(f"✓ Final backup saved: {len(artists_data)} artists to {backup_file}")

# Save to CSV
artists_data_df = pd.DataFrame(artists_data)
artists_data_df.to_csv(output_csv, index=False)
status(f"✓ Saved to CSV: {output_csv} ({len(artists_data)} rows)")

2025-10-24 18:34:40 — No backup found at ../spotify_artists_backup.json; starting fresh
2025-10-24 18:34:40 — Will process 5184 remaining artists (total 5184)


Artists:   0%|          | 0/5184 [00:00<?, ?artist/s]

2025-10-24 18:34:40 — [1/5184] Starting Jeff Tweedy (2rDxtYUzTAYJJE3Bl3Z5IN)
2025-10-24 18:34:48 — Fetched artist info: followers=None
2025-10-24 18:34:57 — Found 21 albums for Jeff Tweedy
2025-10-24 18:34:57 —   [Jeff Tweedy] Fetching tracks for album 1/21
2025-10-24 18:35:05 —   [Jeff Tweedy] Fetching tracks for album 2/21
2025-10-24 18:35:11 —   [Jeff Tweedy] Fetching tracks for album 3/21
2025-10-24 18:35:20 —   [Jeff Tweedy] Fetching tracks for album 4/21
2025-10-24 18:35:29 —   [Jeff Tweedy] Fetching tracks for album 5/21
2025-10-24 18:35:37 —   [Jeff Tweedy] Fetching tracks for album 6/21
2025-10-24 18:35:46 —   [Jeff Tweedy] Fetching tracks for album 7/21
2025-10-24 18:35:48 —   [Jeff Tweedy] Fetching tracks for album 8/21
2025-10-24 18:35:51 —   [Jeff Tweedy] Fetching tracks for album 9/21
2025-10-24 18:35:53 —   [Jeff Tweedy] Fetching tracks for album 10/21
2025-10-24 18:35:55 —   [Jeff Tweedy] Fetching tracks for album 11/21
2025-10-24 18:36:03 —   [Jeff Tweedy] Fetching tra

KeyboardInterrupt: 

In [5]:
# python
import os
import time
import json
from datetime import datetime
from tqdm.notebook import tqdm
import pandas as pd
from spotify_functions import *  # helper functions: get_artist_info, get_artist_albums, get_album_tracks

VERBOSE = True

def status(msg):
    if VERBOSE:
        print(f"{datetime.now().isoformat(sep=' ', timespec='seconds')} — {msg}")

# Files
csv_file = "artist_ids.csv"
backup_file = "../spotify_artists_backup.json"
output_csv = "spotify_artists_data.csv"

# Load CSV
artists_ids_df = pd.read_csv(csv_file)

# Load or initialize backup & processed set (normalize ids to str)
if os.path.exists(backup_file):
    with open(backup_file, 'r') as f:
        artists_data = json.load(f)
    status(f"Resuming with {len(artists_data)} existing artists from {backup_file}")
    processed_ids = {
        str(item.get('artist_id')).strip()
        for item in artists_data
        if isinstance(item, dict) and item.get('artist_id') is not None and not pd.isna(item.get('artist_id'))
    }
else:
    artists_data = []
    processed_ids = set()
    status(f"No backup found at {backup_file}; starting fresh")

total_artists = len(artists_ids_df)
processed_count = len(artists_data)

REQUEST_DELAY = 2.0
SAVE_INTERVAL = 5
MAX_RETRIES = 3
RETRY_BACKOFF = 2.0  # seconds multiplier for retries

status(f"Will process {total_artists - processed_count} remaining artists (total {total_artists})")

# Guard: require access_token to be defined by the user/environment
if 'access_token' not in globals() or not access_token:
    raise RuntimeError("`access_token` not defined. Define it before running or obtain a token from your auth flow.")

with tqdm(total=total_artists, initial=processed_count, desc='Artists', unit='artist') as pbar:
    for index, row in artists_ids_df.iterrows():
        loop_start = time.perf_counter()
        artist_name = row.get('artist', '') or ''
        raw_id = row.get('artist_id')

        # Missing ID -> skip
        if pd.isna(raw_id):
            status(f"Skipping row {index + 1}: {artist_name} — missing `artist_id`")
            pbar.set_postfix({'stage': 'skipped', 'artist': artist_name})
            pbar.update(1)
            continue

        artist_id = str(raw_id).strip()
        if not artist_id:
            status(f"Skipping row {index + 1}: {artist_name} — empty `artist_id` after normalization")
            pbar.set_postfix({'stage': 'skipped', 'artist': artist_name})
            pbar.update(1)
            continue

        # Already processed -> skip
        if artist_id in processed_ids:
            status(f"Skipping [{index + 1}/{total_artists}] {artist_name} ({artist_id}) — already processed")
            pbar.set_postfix({'stage': 'already_processed', 'artist': artist_name})
            pbar.update(1)
            continue

        status(f"[{index + 1}/{total_artists}] Starting {artist_name} ({artist_id})")
        pbar.set_postfix({'stage': 'start', 'artist': artist_name, 'id': artist_id})

        try:
            # Fetch artist info
            pbar.set_postfix({'stage': 'fetching_artist', 'artist': artist_name})
            time.sleep(REQUEST_DELAY)
            artist_info = get_artist_info(artist_id, access_token)

            # normalize artist_info to a dict if wrapped
            if isinstance(artist_info, list) and artist_info:
                artist_info = artist_info[0]
            elif isinstance(artist_info, dict) and isinstance(artist_info.get('artists'), list) and artist_info['artists']:
                artist_info = artist_info['artists'][0]

            followers_count = None
            if isinstance(artist_info, dict):
                followers = artist_info.get('followers', {})
                if isinstance(followers, dict):
                    followers_count = followers.get('total')

            status(f"Fetched artist info: followers={followers_count}")
            pbar.set_postfix({'stage': 'fetched_artist', 'followers': followers_count})

            # Fetch albums
            pbar.set_postfix({'stage': 'fetching_albums', 'artist': artist_name})
            time.sleep(REQUEST_DELAY)
            albums_resp = get_artist_albums(artist_id, access_token)
            if isinstance(albums_resp, dict) and isinstance(albums_resp.get('items'), list):
                albums_list = albums_resp['items']
            elif isinstance(albums_resp, list):
                albums_list = albums_resp
            else:
                albums_list = []

            album_names = [a.get('name') for a in albums_list if isinstance(a, dict)]
            album_release_dates = [a.get('release_date') for a in albums_list if isinstance(a, dict)]
            status(f"Found {len(album_names)} albums for {artist_name}")
            pbar.set_postfix({'stage': 'fetched_albums', 'albums_found': len(album_names)})

            # Fetch tracks per album with local retries and error handling so one bad album doesn't stop the artist
            tracks = []
            for i, album in enumerate(albums_list, start=1):
                album_id = album.get('id') if isinstance(album, dict) else None
                if not album_id:
                    status(f"  [{artist_name}] Skipping album {i}/{len(albums_list)} — missing id")
                    pbar.set_postfix({'stage': 'fetching_album_tracks', 'artist': artist_name, 'album_idx': f"{i}/{len(albums_list)}", 'tracks_collected': len(tracks)})
                    continue

                pbar.set_postfix({
                    'stage': 'fetching_album_tracks',
                    'artist': artist_name,
                    'album_idx': f"{i}/{len(albums_list)}",
                    'tracks_collected': len(tracks)
                })
                status(f"  [{artist_name}] Fetching tracks for album {i}/{len(albums_list)} (id={album_id})")

                tracks_list = []
                for attempt in range(1, MAX_RETRIES + 1):
                    try:
                        time.sleep(REQUEST_DELAY)
                        album_tracks_resp = get_album_tracks(album_id, access_token)
                        if isinstance(album_tracks_resp, dict) and isinstance(album_tracks_resp.get('items'), list):
                            tracks_list = album_tracks_resp['items']
                        elif isinstance(album_tracks_resp, list):
                            tracks_list = album_tracks_resp
                        else:
                            tracks_list = []
                        break
                    except Exception as e2:
                        status(f"    Error fetching tracks for album {i}/{len(albums_list)} (attempt {attempt}): {e2}")
                        if attempt < MAX_RETRIES:
                            wait = RETRY_BACKOFF * attempt
                            status(f"    retrying after {wait}s...")
                            time.sleep(wait)
                        else:
                            status(f"    failed after {MAX_RETRIES} attempts, skipping album {i}/{len(albums_list)}")

                for track in tracks_list:
                    if isinstance(track, dict) and 'name' in track:
                        tracks.append(track['name'])

                # update postfix after each album
                pbar.set_postfix({
                    'stage': 'fetching_album_tracks',
                    'artist': artist_name,
                    'album_idx': f"{i}/{len(albums_list)}",
                    'tracks_collected': len(tracks)
                })

            # Append normalized artist data
            artists_data.append({
                'artist': artist_name,
                'artist_id': artist_id,
                'followers_count': followers_count,
                'albums': album_names,
                'album_release_dates': album_release_dates,
                'tracks': tracks
            })

            processed_ids.add(artist_id)
            processed_count += 1

            elapsed = time.perf_counter() - loop_start
            status(f"Collected {len(album_names)} albums and {len(tracks)} tracks for {artist_name}. Followers: {followers_count}. Time: {elapsed:.2f}s")
            pbar.set_postfix({
                'stage': 'done',
                'artist': artist_name,
                'albums': len(album_names),
                'tracks': len(tracks),
                'followers': followers_count
            })

            # Periodic save
            if processed_count % SAVE_INTERVAL == 0:
                with open(backup_file, 'w') as f:
                    json.dump(artists_data, f, indent=2)
                status(f"✓ Backup saved ({processed_count} artists) to {backup_file}")
                pbar.set_postfix({'stage': 'backup_saved', 'saved': processed_count})

            pbar.update(1)

        except Exception as e:
            status(f"Error processing {artist_name}: {e}")
            pbar.set_postfix({'stage': 'error', 'artist': artist_name, 'error': str(e)[:120]})
            # Save backup on error
            with open(backup_file, 'w') as f:
                json.dump(artists_data, f, indent=2)
            status(f"✓ Backup saved after error ({len(artists_data)} artists) to {backup_file}")

            # Rate limit handling
            if "429" in str(e) or "rate limit" in str(e).lower():
                wait_time = 60
                status(f"Rate limit detected. Waiting {wait_time}s...")
                time.sleep(wait_time)

            pbar.update(1)

# Final save
with open(backup_file, 'w') as f:
    json.dump(artists_data, f, indent=2)
status(f"✓ Final backup saved: {len(artists_data)} artists to {backup_file}")

# Save to CSV
artists_data_df = pd.DataFrame(artists_data)
artists_data_df.to_csv(output_csv, index=False)
status(f"✓ Saved to CSV: {output_csv} ({len(artists_data)} rows)")

2025-10-24 18:43:47 — No backup found at ../spotify_artists_backup.json; starting fresh
2025-10-24 18:43:47 — Will process 5184 remaining artists (total 5184)


Artists:   0%|          | 0/5184 [00:00<?, ?artist/s]

2025-10-24 18:43:47 — [1/5184] Starting Jeff Tweedy (2rDxtYUzTAYJJE3Bl3Z5IN)
2025-10-24 18:43:50 — Fetched artist info: followers=None
2025-10-24 18:43:52 — Found 21 albums for Jeff Tweedy
2025-10-24 18:43:52 —   [Jeff Tweedy] Fetching tracks for album 1/21 (id=4nczkBnxvOZNewjhxws5q6)
2025-10-24 18:43:55 —   [Jeff Tweedy] Fetching tracks for album 2/21 (id=1CxhjvovjuPCVqbheI9Wq8)
2025-10-24 18:43:57 —   [Jeff Tweedy] Fetching tracks for album 3/21 (id=4luinFPZgIsSXjsKqb42LI)
2025-10-24 18:44:05 —   [Jeff Tweedy] Fetching tracks for album 4/21 (id=1XxXW0gpZkFw9krv7DAILO)
2025-10-24 18:44:13 —   [Jeff Tweedy] Fetching tracks for album 5/21 (id=0x5Bsyh5DbFSChrkSIE0sn)
2025-10-24 18:44:22 —   [Jeff Tweedy] Fetching tracks for album 6/21 (id=6wIkxlcWROUTKlwU6vrJL9)
2025-10-24 18:44:30 —   [Jeff Tweedy] Fetching tracks for album 7/21 (id=6dsqls3emhakKWVRPHzhtI)
2025-10-24 18:44:38 —   [Jeff Tweedy] Fetching tracks for album 8/21 (id=7n9A156DrLyOTTmU9ybKmd)
2025-10-24 18:44:47 —   [Jeff Tweed

KeyboardInterrupt: 